In [1]:
!echo $http_proxy

http://squid-compute101:3128


In [2]:
import comet_ml

comet_ml.login(project_name="Random_search")

In [3]:
import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical

2024-11-26 13:01:46.438839: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255


In [5]:
print(x_train.shape)

(60000, 28, 28, 1)


In [6]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [7]:
def build_model_graph(experiment):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28, 1)))  # Flatten layer to reshape (28,28,1) to (784,)
    model.add(
        Dense(
            experiment.get_parameter("first_layer_units"),
            activation="sigmoid",
            input_shape=(784,),
        )
    )
    model.add(Dense(128, activation="sigmoid"))
    model.add(Dense(128, activation="sigmoid"))
    model.add(Dense(10, activation="softmax"))
    model.compile(
        loss="categorical_crossentropy",
        optimizer=RMSprop(),
        metrics=["accuracy"],
    )
    return model


def train(experiment, model, x_train, y_train, x_test, y_test):
    model.fit(
        x_train,
        y_train,
        batch_size=experiment.get_parameter("batch_size"),
        epochs=experiment.get_parameter("epochs"),
        validation_data=(x_test, y_test),
    )


def evaluate(experiment, model, x_test, y_test):
    score = model.evaluate(x_test, y_test, verbose=0)
    print("Score %s", score)



In [8]:
# The optimization config:
config = {
    "algorithm": "random",  # Change to random search
    "name": "Optimize MNIST Network with Random Search",
    "spec": {"maxCombo": 15, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "first_layer_units": {
            "type": "integer",
            "min": 8,
            "max": 64,
        },
        "batch_size": {
            "type": "discrete",
            "values": [8, 16, 32],
        },
        "learning_rate": {
            "type": "float",
            "min": 0.0001,
            "max": 0.01,
        },
    },
    "trials": 1,
}
opt = comet_ml.Optimizer(config)

COMET INFO: 61fe9926a0b2429ab77580bbf796ca33
COMET INFO: Using optimizer config: {'algorithm': 'random', 'configSpaceSize': 300, 'endTime': None, 'id': '61fe9926a0b2429ab77580bbf796ca33', 'lastUpdateTime': None, 'maxCombo': 15, 'name': 'Optimize MNIST Network with Random Search', 'parameters': {'batch_size': {'type': 'discrete', 'values': [8, 16, 32]}, 'first_layer_units': {'max': 64, 'min': 8, 'scalingType': 'uniform', 'type': 'integer'}, 'learning_rate': {'max': 0.01, 'min': 0.0001, 'scalingType': 'uniform', 'type': 'float'}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 15, 'metric': 'loss', 'minSampleSize': 100, 'objective': 'minimize', 'retryAssignLimit': 0, 'retryLimit': 1000}, 'startTime': 6051966556, 'state': {'mode': None, 'seed': 3619596592, 'sequence': [], 'sequence_i': 0, 'sequence_pid': None, 'sequence_retry': 0, 'sequence_retry_count': 0}, 'status': 'running', 'suggestion_count': 0, 'trials': 1, 'version': '2.1.2'}


In [9]:
for experiment in opt.get_experiments():
    # Log parameters, or others:
    experiment.log_parameter("epochs", 10)

    # Build the model:
    model = build_model_graph(experiment)

    # Train it:
    train(experiment, model, x_train, y_train, x_test, y_test)

    # How well did it do?
    evaluate(experiment, model, x_test, y_test)

    # Optionally, end the experiment:
    experiment.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/nast69/random-search/b74e0b9b928d4ebb870facb1ff814ae6

/lustre06/project/6078700/nast/comet/ENV/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/lustre06/project/6078700/nast/comet/ENV/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-26 13:05:50.

Epoch 1/10


I0000 00:00:1732644355.220520 1136148 service.cc:145] XLA service 0x14d7e8014b50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732644355.220697 1136148 service.cc:153]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute Capability 8.0
2024-11-26 13:05:55.451969: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-26 13:05:55.583672: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8905
2024-11-26 13:05:55.618604: W external/local_xla/xla/service/gpu/triton_autotuner.cc:660] Compiling 71 configs for 7 fusions on a single thread.


 186/7500 ━━━━━━━━━━━━━━━━━━━━ 5s 818us/step - accuracy: 0.1547 - loss: 2.3049

I0000 00:00:1732644362.095296 1136148 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

2024-11-26 13:18:23.680364: W external/local_xla/xla/service/gpu/triton_autotuner.cc:660] Compiling 41 configs for 3 fusions on a single thread.


3750/3750 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.5643 - loss: 1.2567 - val_accuracy: 0.8830 - val_loss: 0.4033
Epoch 2/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 3s 890us/step - accuracy: 0.9016 - loss: 0.3448 - val_accuracy: 0.9195 - val_loss: 0.2711
Epoch 3/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 3s 886us/step - accuracy: 0.9214 - loss: 0.2720 - val_accuracy: 0.9276 - val_loss: 0.2478
Epoch 4/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 5s 884us/step - accuracy: 0.9325 - loss: 0.2347 - val_accuracy: 0.9355 - val_loss: 0.2257
Epoch 5/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 3s 890us/step - accuracy: 0.9377 - loss: 0.2164 - val_accuracy: 0.9369 - val_loss: 0.2213
Epoch 6/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 3s 893us/step - accuracy: 0.9406 - loss: 0.2014 - val_accuracy: 0.9424 - val_loss: 0.2083
Epoch 7/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 3s 885us/step - accuracy: 0.9455 - loss: 0.1927 - val_accuracy: 0.9423 - val_loss: 0.2106
Epoch 8/10
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 5s 896us/step - accuracy: 0.9465 - loss: 0.1875

2024-11-26 13:19:06.004338: W external/local_xla/xla/service/gpu/triton_autotuner.cc:656] Compiling 16 configs for triton_gemm_dot.50 on a single thread.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : federal_turtle_4187
COMET INFO:     url                   : https://www.comet.com/nast69/random-search/c4b91fc5a0f148c4a1536daa40f7bb20
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [10]                 : (0.7555333375930786, 0.9492999911308289)
COMET INFO:     batch_accuracy [3750]         : (0.0625, 1.0)
COMET INFO:     batch_loss [3750]             : (0.005507225636392832, 2.504549264907837)
COMET INFO:     epoch_duration [10]           : (3.3392275869991863, 7.11737994199938

Score %s [0.19504310190677643, 0.944599986076355]


COMET INFO: Uploading 75 metrics, params and output messages
COMET INFO: Optimizer search 61fe9926a0b2429ab77580bbf796ca33 has completed


In [ ]:
# Configuration for the optimizer
config = {
    "algorithm": "random",  # Random search for hyperparameter optimization
    "name": "Optimize MNIST Network with Random Search",
    "spec": {"maxCombo": 15, "objective": "minimize", "metric": "loss"},  # Optimization details
    "parameters": {  # Hyperparameter definitions
        "first_layer_units": {"type": "integer", "min": 8, "max": 64},  # Number of units in the first layer
        "batch_size": {"type": "discrete", "values": [8, 16, 32]},  # Batch size options
        "learning_rate": {"type": "float", "min": 0.0001, "max": 0.01},  # Learning rate range
    },
    "trials": 1,  # Number of trials to run
}

# Initialize the optimizer with the given configuration
opt = comet_ml.Optimizer(config)

# Iterate over experiments generated by the optimizer
for experiment in opt.get_experiments():
    experiment.log_parameter("epochs", 10)  # Log fixed parameter: number of epochs

    model = build_model_graph(experiment)  # Build the model with the experiment's parameters
    train(experiment, model, x_train, y_train, x_test, y_test)  # Train the model on the dataset
    evaluate(experiment, model, x_test, y_test)  # Evaluate the model's performance

    experiment.end()  # End the experiment (optional but recommended)
